# Setup

In [ ]:
%matplotlib inline

import numpy as np, pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path
import seaborn as sns 
import sklearn
from sklearn import datasets

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
if 'google.colab' in str(get_ipython()):
    print('The notebook is running on Colab. colab=True.')
    colab=True
else:
    print('The notebook is not running on Colab. colab=False.')
    colab=False

# Data

In [ ]:
NB_DIR = Path.cwd()
DATA = NB_DIR/'data'
DATA.mkdir(exist_ok=True)

Vi bruker datasettet fra innlevering 1:

In [ ]:
if colab:
    df = pd.read_csv('https://www.dropbox.com/s/ctlniradiv1i5id/assignment1-train.csv?dl=1')

In [ ]:
if not colab:
    df = pd.read_csv(DATA/'assignment1-train.csv')

# Imputering

> Hvordan håndtere manglende verdier? 

1. Først må de **detekteres**. I praksis kan manglende verdier være kodet på omtrent hvilken som helst måte. Ofte bruker man blanke verdier, placeholder-verdier (f.eks. -1) eller NaNs (not-a-number).

2. Deretter kan man _imputere_. Det vil si, fylle inn manglende data

Historien vi skal fortelle om dette har behov for at vi er i en maskinlærings-situasjon med X, y og trenings- og test-data:

In [ ]:
X, y = df.drop('target', axis=1), df.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## 1. Detektere manglende verdier

Det første man typisk undersøker er hvorvidt det er NaN-verdier i datasettet. Hvis en arbeider med en Pandas dataframe er dette enkelt: 

In [ ]:
X_train.isna()

In [ ]:
X_train.isna().sum()

In [ ]:
# For å tvinge Pandas til å vise alle radene i datasettet: 
with pd.option_context('display.max_rows', None): 
    print(X_train.isna().sum())

In [ ]:
plt.figure(figsize=(16,8))
X_train.isna().sum().plot(kind='bar')
plt.show()

Men manglende verdier kan være kodet annerledes. En måte å finne disse er å se på hvilke verdier som finnes i datasettet. 

I vårt tilfelle er alle features kodet som flyt-tall. Altså kan manglende verdier ikke være reprentert som strenger (f.eks. som tomme strenger " "):

In [ ]:
X_train.info()

Hvis features ikke har for mange ulike verdier er det nyttig å telle opp antall instanser som har hver verdi:

In [ ]:
X_train['f_0'].value_counts()

In [ ]:
for feature in X_train.columns[1:]:   # Vi tar ikke med id-søylen
    print(f"Value count for {feature}")
    with pd.option_context('display.max_rows', None): 
        print(X_train[feature].value_counts())
    print("#"*40)

Det ser i vårt tilfelle ut som at alle manglende verdier er kodet som NaN-verdier. 

## 2. Imputere

Det neste blir å erstatte disse NaN-verdiene. Som vi har sett på er det mange ulike strategier. 

### Enkle strategier

Vi starter med noen enkle strategier: erstatte verdiene ved å bruke gjennomsnitt, median eller den hyppigste verdien

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
?SimpleImputer

In [ ]:
imp = SimpleImputer(strategy='mean')

Merk at man kan bruke samme strategi for hver feature, eller ulike strategier for ulike features:

Her er samme strategi brukt for alle features:

In [ ]:
X_train_imputed = imp.fit_transform(X_train)
# Reprenter som en dataframe:
X_train_imputed = pd.DataFrame(data=X_train_imputed, columns=X_train.columns)

In [ ]:
with pd.option_context('display.max_rows', None):
    print(X_train_imputed.isna().sum())

Her er feature-spesifikke strategier per feature:

In [ ]:
# Vi lager en kopi slik at vi enkelt kan reversere operasjonen:
X_train_copy = X_train.copy()

In [ ]:
features = ['f_0', 'f_1']
imp = SimpleImputer(strategy='mean')
# Transformer de valgte features:
X_train_imp = imp.fit_transform(X_train_copy[features])
# Erstatt søylene med de nye verdiene
X_train_copy[features] = X_train_imp

In [ ]:
features = ['f_6', 'f_7']
imp = SimpleImputer(strategy='median')
# Transformer de valgte features:
X_train_imp = imp.fit_transform(X_train_copy[features])
# Erstatt søylene med de nye verdiene
X_train_copy[features] = X_train_imp

In [ ]:
features = ['f_8']
imp = SimpleImputer(strategy='most_frequent')
# Transformer de valgte features:
X_train_imp = imp.fit_transform(X_train_copy[features])
# Erstatt søylene med de nye verdiene
X_train_copy[features] = X_train_imp

In [ ]:
X_train[40:80]

In [ ]:
X_train_copy[40:80]

### Mer avanserte strategier

En kan være mer spissfindig enn å imputere verdier ved å bruke verdiene til alle instanser. Man kan for eksempel finne hvilke verdier en skal erstatte med ved å bruke kun _lignende_ instanser, istedenfor alle. 

En strategi for dette er å trene en modell som kan gruppere lignende instanser. Et eksempel på dette er såkalte **K næreste nabo** eller KNN.  

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/850px-KnnClassification.svg.png'>

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
?KNNImputer

In [ ]:
knn_imp = KNNImputer()

Det tar en stund å trene modellen så vi bruker den her kun på to features:

In [ ]:
X_train_copy = X_train.copy()

In [ ]:
X_train[40:50]

In [ ]:
features = ['f_0', 'f_1']
imp = KNNImputer()
# Transformer de valgte features:
X_train_imp = imp.fit_transform(X_train_copy[features])
# Erstatt søylene med de nye verdiene
X_train_copy[features] = X_train_imp

In [ ]:
X_train_copy[40:50]

### En annen strategi: tren en modell til å imputere

En annen strategi er å trene en regresjonsmodell, for eksempel en RandomForestRegressor, til å predikere manglende verdier fra verdiene som ikke mangler. 

Dette kan gjøres iterativt: i hvert steg velges en feature-søyle til å gi outputs `y`. Deretter trenes en modell på alle features uten manglende verdier til å predikere `y` ved å bruke de instansene der en kjenner verdiene av `y`. Denne modellen kan så brukes til å fylle inn. Dette kan en iterere over alle søylene til en ikke lenger har manglende verdier, og man kan gjøre det om og om igjen til en når et maksimalt antall iterasjoner. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()

In [ ]:
#?IterativeImputer

In [ ]:
X_train_copy = X_train.copy()

In [ ]:
X_train_copy[40:50]

In [ ]:
features = ['f_0', 'f_1']
imp = IterativeImputer(estimator=estimator)
# Transformer de valgte features:
X_train_imp = imp.fit_transform(X_train_copy[features])
# Erstatt søylene med de nye verdiene
X_train_copy[features] = X_train_imp

In [ ]:
X_train_copy[40:50]

# Imputering for test-settet

***OBS: Her ligger det en litt skjult fare!*** Pass på å finne ut hvilke verdier en skal putte inn fra _treningsdata_, ikke fra testdata!

In [ ]:
imp = SimpleImputer(strategy='mean')

In [ ]:
X_train_imputed = imp.fit_transform(X_train)
X_test_imputed = imp.transform(X_test)

> ***Hvorfor bruker man `.fit_transform` på treningssettet, men `.transform` på test-settet?***